# Getting Started With `GeoPandas`

## Setup & Environment

First, we'll need to install the `GeoPandas`.

Installing and configuring `Geopandas` requires creating a new Python environment.

A few resources that can get folks started:
  
- Anaconda
  * Tanish Gupta, "[Fastest Way to Install Geopandas in Jupyter Notebooks](https://medium.com/analytics-vidhya/fastest-way-to-install-geopandas-in-jupyter-notebook-on-windows-8f734e11fa2b)" *Analytics Vidhya* (6 December 2020)
  * Anaconda, "[conda-forge packages, geopandas](https://anaconda.org/conda-forge/geopandas)" *Anaconda documentation*
  * GeoPandas, "[Installation](https://geopandas.org/getting_started/install.html)" *GeoPandas documentation*
- Google CoLab
  * Abdishakur Hassan, Jupyter notebook on using `geopandas` in Google CoLab, from "[Geographic data science tutorials with Python](https://github.com/shakasom/GDS)" *GitHub repository*
    * [Google CoLab](https://colab.research.google.com/github/shakasom/GDS/blob/master/Part1%20-%20Introduction.ipynb)
    * [GitHub](https://github.com/shakasom/GDS/blob/master/Part1%20-%20Introduction.ipynb)

Additional `GeoPandas` resources:
- Jonathan Soma, "[Mapping with geopandas](https://jonathansoma.com/lede/foundations-2017/classes/geopandas/mapping-with-geopandas/)" from 2017 "[Foundations of Computing](https://jonathansoma.com/lede/foundations-2017/)" course, Columbia Graduate School of Journalism
- CoderzColumn, "[Plotting Static Maps with geopandas](https://coderzcolumn.com/tutorials/data-science/plotting-static-maps-with-geopandas-working-with-geospatial-data)" *CoderzColumn* (11 March 2020)
- GeoPandas, "[Plotting with Geoplot and GeoPandas](https://geopandas.org/gallery/plotting_with_geoplot.html)" *GeoPandas documentation*

In [ ]:
# if working in Google Colab
!pip install geopandas

In [ ]:
# import statements
import pandas as pd, geopandas as gpd, json, requests

When possible, loading geospatial data (especially polygon data) through `GeoPandas` will simplify other workflows.

What distinguishes a `GeoDataFrame` from a standard `DataFrame`? The all important `geometry` column.

For more on data structures in `GeoPandas`:
- [GeoPandas documentation](https://geopandas.org/en/stable/docs/user_guide/data_structures.html)
- [Spatial analysis with Python tutorial](https://sustainability-gis.readthedocs.io/en/latest/lessons/L1/intro-to-python-geostack.html)

## Dataset #1

The first dataset we'll use in this chapter is data about [City of South Bend parks](https://data-southbend.opendata.arcgis.com/datasets/SouthBend::parks-locations-and-features/about).

An API call to bring that data into Python:

In [ ]:
import pandas as pd, json, requests # import statements
r = requests.get('https://services1.arcgis.com/0n2NelSAfR7gTkr1/arcgis/rest/services/Parks_Locations_and_Features/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson') # load page
d = r.json() # store as json object

data = [] # empty list for data

for i in d['features']: # iterate over list
  data.append(i['properties']) # isolate value, append to list

df = pd.DataFrame(data) # create dataframe
df.info() # show output

We'll need `latitude` and `longitude`, and those values are currently buried in the `Location_1` column. So we'll start there by splitting out that column on the `\n` character.

In [ ]:
df[['Address', 'City', 'LatLon']] = df['Location_1'].str.split(r'\n', expand=True) # split column
df.head() # show output

We're closer!

The next step is breaking out the `latitude` and `longitude` values, and removing the `()` characters.

In [ ]:
df['LatLon'] = df['LatLon'].str.replace('[()]', '') # remove parentheses
df[['Latitude', 'Longitude']] = df['LatLon'].str.split(',', expand=True) # split column
df.head() # show output

Now we can convert this to a `GeoDataFrame`.

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Latitude, df.Longitude), crs="EPSG:4326") # convert to gdf
gdf.to_file("parks.json", driver='GeoJSON')
gdf.info() # inspect output

Now we have the all-important `geometry` column for our first dataset.

## Polygon Data

For our second dataset, let's work with the St. Joseph County zip code boundary file.
- [Link to download](https://sjcgis-stjocogis.hub.arcgis.com/datasets/stjocogis::zip-code-boundaries-3/about)
  * *Note: I've renamed this file `zip.geojson`.*

In [ ]:
gdf = gpd.read_file("zip.geojson") # load file
gdf.head() # show geo dataframe head

,FID,ZIP,City_Town,Shape__Area,Shape__Length,geometry
0,1,46506,Bremen,1.836947e+08,71554.432719,"POLYGON ((-86.09920 41.50815, -86.09914 41.500..."
1,2,46530,Granger,1.074689e+08,59262.193560,"POLYGON ((-86.06268 41.73220, -86.06268 41.731..."
2,3,46536,Lakeville,1.207107e+08,50337.989555,"POLYGON ((-86.24495 41.56449, -86.24457 41.563..."
3,4,46544,Mishawaka,1.790765e+08,78110.454324,"POLYGON ((-86.06125 41.60846, -86.06126 41.608..."
4,5,46545,Mishawaka,7.911272e+07,56893.621236,"POLYGON ((-86.09635 41.72453, -86.09639 41.719..."


Let's connect those polygons with educational outcome attribute data from the American Community Survey with the zip code boundary file.
- <a href="https://data.census.gov/table/ACSST5Y2022.S1501?t=Education:Educational Attainment&g=050XX00US18141,18141$8600000&moe=false">Link to the ACS dataset</a>
  * *Note: I'm working with 2022's 5 year estimate, which I've renamed `data.csv`.*

In [ ]:
df = pd.read_csv("data.csv") # load attribute data
df.columns = df.columns.str.split("!!", 2, expand=True) # split column headers into multi-level index based on separator
df = df.T # transpose dataframe
header = df.iloc[0] # isolate first row to be new header
df = df[1:] # subset dataframe (everything past the first row)
df.columns = header # reassign headers
df = df.reset_index() # reset the index
df.columns.values[0] = 'Area' # rename columns
df.columns.values[1] = 'Coverage'
df.columns.values[2] = 'Type'
df['Area'] = df['Area'].str.replace("ZCTA5 ", "") # standardize zip code column
df # show output

<ipython-input-31-fa2a43ce9c61>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df.columns = df.columns.str.split("!!", 2, expand=True) # split column headers into multi-level index based on separator


"(Label (Grouping), nan, nan)",Area,Coverage,Type,AGE BY EDUCATIONAL ATTAINMENT,Population 18 to 24 years,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher,Population 25 years and over,...,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher,MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS),Population 25 years and over with earnings,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree,Graduate or professional degree
0,"St. Joseph County, Indiana",Total,Estimate,NaN,"31,651","2,781","11,312","13,760","3,798","177,621",...,(X),(X),(X),NaN,"43,401","28,431","35,439","40,901","51,191","70,582"
1,"St. Joseph County, Indiana",Male,Estimate,NaN,"15,346","1,580","6,063","6,037","1,666","85,726",...,(X),(X),(X),NaN,"51,874","32,554","44,936","51,189","64,026","85,742"
2,"St. Joseph County, Indiana",Female,Estimate,NaN,"16,305","1,201","5,249","7,723","2,132","91,895",...,(X),(X),(X),NaN,"35,489","24,167","27,320","33,550","43,238","59,452"
3,46506,Total,Estimate,NaN,773,256,263,240,14,"6,667",...,(X),(X),(X),NaN,"46,271","34,290","45,855","43,802","53,264",-
4,46506,Male,Estimate,NaN,359,94,192,73,0,"3,221",...,(X),(X),(X),NaN,"59,384","34,321","53,333","68,245","85,303",-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,46635,Male,Estimate,NaN,132,9,110,13,0,"2,177",...,(X),(X),(X),NaN,"50,234","39,219","55,801","49,879","45,658","75,625"
74,46635,Female,Estimate,NaN,334,193,89,42,10,"2,579",...,(X),(X),(X),NaN,"42,725",-,-,"38,750","45,556","76,154"
75,46637,Total,Estimate,NaN,"1,956",114,566,782,494,"11,516",...,(X),(X),(X),NaN,"45,659","45,724","34,778","39,607","66,181","53,140"
76,46637,Male,Estimate,NaN,"1,091",31,373,427,260,"5,669",...,(X),(X),(X),NaN,"56,035","63,088","45,677","48,419","70,833","72,875"


Now we can use `GeoPandas` to merge these datasets.

In [ ]:
merged = gdf.merge(df, left_on="ZIP", right_on="Area") # merged attribute and geospatial data
merged # show merged geodataframe